# Problem 1

**Xuefeng Pei**

## (a)

For a single column, we have four rows. Which means we have 2 or less pebbles on this coumn.  
0 pebbels -> 1 pattern \[0, 0, 0, 0\]  
1 pebbels -> 4 patterns \[1, 0, 0, 0\], \[0, 1, 0, 0\], \[0, 0, 1, 0\], \[0, 0, 0, 1\]   
2 pebbels -> 3 patterns \[1, 0, 1, 0\],  \[0, 1, 0, 1\], \[1, 0, 0, 1\]  

So the number of patterns should be 1 + 4 + 3 = **8**

## (b)

### Approach

Clearly, there are at most **8** possible patterns for each column. Let F(i, c) be the maximum sum of the first **c** columns with pattern **i**. Then:   
**F(i, c) = max{ Value(i) + F(j, c-1) }**, where i is from 0 to 7 and i, j are a pair of compatiable patterns.  

And then we just need to keep a track of the optimal placement and return the result.

### Code

For the following code, I'll use pattern number to match the patterns above and return a list of pattern numbers to represent the placement.  

pattern 0:  \[0, 0, 0, 0\]  
pattern 1: \[1, 0, 0, 0\]  
pattern 2: \[0, 1, 0, 0\]  
pattern 3: \[0, 0, 1, 0\]  
pattern 4: \[0, 0, 0, 1\]  
pattern 5: \[1, 0, 1, 0\]  
pattern 6: \[0, 1, 0, 1\]  
pattern 7: \[1, 0, 0, 1\] 

Apparently, the compatiability is as follows:  
0 -> 1, 2, 3, 4, 5, 6, 7  
1 -> 0, 2, 3, 4, 6  
2 -> 0, 1, 3, 4, 5, 7  
3 -> 0, 1, 2, 4, 6, 7  
4 -> 0, 1, 2, 3, 5  
5 -> 0, 2, 4, 6  
6 -> 0, 1, 3, 5  
7 -> 0, 2, 3  

In [1]:
from typing import List

In [4]:
def optimal_placement(board: List[List[int]], n: int) -> List[int]:
    def value(c, i):
        if i == 0:
            return 0
        elif i == 1:
            return board[0][c]
        elif i == 2:
            return board[1][c]
        elif i == 3:
            return board[2][c]
        elif i == 4:
            return board[3][c]
        elif i == 5:
            return board[0][c] + board[2][c]
        elif i == 6:
            return board[1][c] + board[3][c]
        elif i == 7:
            return board[0][c] + board[3][c]
        
    compatiability = [(1, 2, 3, 4, 5, 6, 7), 
                      (0, 2, 3, 4, 6), 
                      (0, 1, 3, 4, 5, 7), 
                      (0, 1, 2, 4, 6, 7), 
                      (0, 1, 2, 3, 5), 
                      (0, 2, 4, 6), 
                      (0, 1, 3, 5), 
                      (0, 2, 3)]
    res = [[[] for _ in range(8)] for _ in range(n)]
    dp = [[0] * 8 for _ in range(n)]
    for i in range(8):
        dp[0][i] = value(c, i)
        res[0][i].append(i)
    
    for c in range(1, n):
        dp[c], res[c] = dp[c-1], res[c-1]
        for i in range(8):
            candidate = 0
            for j in range(8):
                if j in compatiability[i] and dp[c-1][j] + value(c, i) > dp[c][i]:
                    dp[c][i] = dp[c-1][j] + value(c, i)
                    candidate = j
            res[c][i].append(candidate)
    
    max_p = dp[n-1].index(max(dp[n-1]))
    return res[n-1][max_p]

### Analysis:

Time complexity: **O(n)** as the inner two for loops are constant time.  
Space complexity: **O(n^2)** as we use a **res** array to store all the pattern combinations.

<br/><br/>

# Problem 2

### Approach

Let F(i, j) be the set of results of String **s** from index i to j.  
F(i, j) = UNION {  
  product(p, q), for p in F(i, k), for q in F(k, j), k from i to j  
}

Base case: F(i, i) = s\[i\]   
For this problem, I'll use top down with memoization to solve it.

### Code

In [17]:
def can_form_a(s: str) -> bool:
    products = {
        ('a', 'a'): 'b',
        ('a', 'b'): 'b',
        ('a', 'c'): 'a',
        ('b', 'a'): 'c',
        ('b', 'b'): 'b',
        ('b', 'c'): 'a',
        ('c', 'a'): 'a',
        ('c', 'b'): 'c',        
        ('c', 'c'): 'c',        
    }
    dp = [[None] * len(s) for _ in range(len(s))]
    
    def helper(i: int, j: int) -> set:
        if i == j:
            return set(s[i])
        if dp[i][j] is not None:
            return dp[i][j]
        res = set()
        for k in range(i, j):
            for p in helper(i, k):
                for q in helper(k+1, j):
                    res.add(products[p, q])
        dp[i][j] = res
        return res
    
    return 'a' in helper(0, len(s)-1)

### Analysis

Time complexity: O(n^3) as we are actually filling a table of n^2 size, for each result, we need O(n) time to calculate.  
Space complexity: O(n^2) for the dp table.

<br/><br/>

# Problem 3

### Approach

Let **F(n, k)** be the probability of **k** heads with the first **n** coins. Now, let's throw the **nth** coin, it's either head or tail, so there are two cases where we can get **k** heads:
- nth coin is head: p\[n-1\] * F(n-1, k-1)
- nth coin is tail: (1 - p\[n-1\]) * F(n-1, k)
- F(n, 0) = (1 - p\[n-1\]) * F(n-1, 0)
- F(0, 0) = 1
- F(0, k) = 0 if k > 0

So, F(n, k) = p\[n-1\] * F(n-1, k-1) + (1 - p\[n-1\]) * F(n-1, k)  

### Code

In [317]:
def count_head(n, k, p: List[int]) -> int:
    if k > n:
        return 0
    dp = [[0] * (k+1) for _ in range(2)]
    dp[0][0] = 1
    
    for i in range(1, n+1):
        curr, prev = i % 2, (i + 1) % 2
        dp[curr][0] = (1 - p[i-1]) * dp[prev][0]
        for j in range(1, min(n, k)+1):
            dp[curr][j] = p[i-1] * dp[prev][j-1] + (1 - p[i-1]) * dp[prev][j]
    print(dp)
    return dp[n%2][k]

### Analysis

Time complexity: O(n^2) as we used two for loops and k is at most n.  
Space complexity: O(n) as we optimized the dp array to 2 * n.

<br/><br/>

# Problem 4

## (a)

Counter example: \[3, 100, 2, 1\]  
Player 1 choose 3  
Player 2 choose 100  
Player 1 choose 2  
Player 2 choose 1  

Player 1 total sum is 5, and Player 2 total sum is 101. So it's not a optimal choice.

## (b)

### Approach

Let f(i, j) be the maximum number we can get between i-th to j-th numbers.  
There are two cases, we either take **i** or **j**, then in each case, the opponent also have two choices, but since he'll also try to maximize his score, we are left of the minimum possible score. Therefore:
- f(i, j) = max(cards\[i\] + min(f(i+2, j), f(i+1, j-1)), cards\[j\] + min(f(i, j-2), f(i+1, j-1)))
- f(i, j) = 0, if i > j
- f(i, j) = cards[i] if i == j
- f(i, j) = max(cards[i], cards[j]), if i + 1 == j

Now as we can calculate our maximum score at each \[i, j\], we can know which one to pick given a certain **i, j**.  
So I'll try to precompute a table to store which one **index** to take given a **i, j**.

In [288]:
def optimal_move(cards: List[int]) -> List[List[int]]:
    res = [[0] * len(cards) for _ in range(len(cards))]
    dp = [[None] * len(cards) for _ in range(len(cards))]
    def helper(i: int, j: int):
        if i > j:
            return 0
        if i == j:
            res[i][j] = i
            return cards[i]
        if i + 1 == j:
            res[i][j] = i if cards[i] > cards[j] else j
            return max(cards[i], cards[j])
        if dp[i][j] is not None:
            return dp[i][j]
        a = cards[i] + min(helper(i+2, j), helper(i+1, j-1))
        b = cards[j] + min(helper(i, j-2), helper(i+1, j-1))
        dp[i][j] = max(a, b)
        res[i][j] = i if a > b else j
        return dp[i][j]
    helper(0, len(cards)-1)
    return res

### Analysis

Time complexity: O(n^2) as we are trying to fill a dp table of size n^2.  
Space complexity: O(n^2) for the dp and result table and recursive call stack.

<br/><br/>

# Problem 5

### Approach

Let's use **P(i, j)** to indicate the probability of A win the game when A has **i** games left to win and B has **j** games left to win. Each time, either A wins or B wins, so we can easily get the following formular:
- Base case: P(0, j) = 1  P(i, 0) = 0  
- Normal case P(i, j) = 1/2(P(i-1, j) + P(i, j-1))  

Example:  
P(1, 3) = 1/2(P(0, 3) + P(1, 2))  
        = 1/2(1 + 1/2(P(0, 2) + P(1, 1)))  
        = 1/2(1 + 1/2(1 + 1/2(P(0, 1) + P(1, 0)))  
        = 1/2(1 + 1/2(1 + 1/2(1 + 0)))  
        = 1/2(1 + 1/2 * 3/2)  
        = 1/2 * (1 + 3/4)  
        = 1/2 * 7/4 = 7/8  

### Code

In [301]:
def cal_a_win(i: int, j: int, n: int) -> float:
    i, j = n - i, n - j
    dp = [[0] * (j + 1) for _ in range(2)]
    for c in range(j+1):
        dp[0][c] = 1
        
    for r in range(1, i+1):
        curr, prev = r % 2, (r + 1) % 2
        for c in range(1, j+1):
            dp[curr][c] = 1/2 * (dp[prev][c] + dp[curr][c-1])
            
    return dp[i%2][j]

### Analysis

Time complexity: O(n^2) as we initial i, j value could be 0.  
Space complexity: O(n) as we optimized our dp array to 2 * n at most.  

<br/><br/>

# Problem 6

### Approach

Let F(i) indicates whether we can make change of value i.  
- F(i) = F(i) or F(i-coin), for coin in all coins  
- F(0) = True  
- F(i) = False if i < 0

However, instead of top-down approach, I think bottom-up is simpler for this problem.

### Code

In [163]:
def make_change1(value: int, coins: List[int]) -> bool:
    dp = [False] * (value + 1)
    dp[0] = True
    for coin in coins:
        for x in range(coin, value+1):
            dp[x] = dp[x] or dp[x-coin]
    
    return dp[value]

### Analysis

Time complexity: O(nv) for the two for loops.  
Space complexity: O(v) for the dp array.

<br/><br/>

# Problem 7

### Appraoch

This problem is similar to the 0-1 knapsack problem. For each coin, we either take it or not.  
Let F(i, j) be the boolean value whether we can form value **j** with the first **i** coins:
- F(i, j) = F(i-1, j) or F(i-1, j-coins\[i\]) if j >= coins\[i\]
- F(i, 0) = True  

So we build up the table with bottom-up approach and also we can optimize the space complexity.

### Code

In [162]:
def make_change2(value: int, coins: List[int]) -> bool:
    dp = [[False] * (value+1) for _ in range(2)]
    for i in range(len(dp)):
        dp[i][0] = True
    
    for i in range(1, len(coins) + 1):
        curr, prev = i % 2, (i + 1) % 2
        for j in range(1, value+1):
            dp[curr][j] = dp[prev][j]
            if j - coins[i-1] >= 0:
                dp[curr][j] = dp[curr][j] or dp[prev][j-coins[i-1]]
                
    return dp[len(coins)%2][value]

### Analysis:

Time complexity: O(nv) for the two for loops.  
Space complexity: O(v) for the 2 * v dp array.

<br/><br/>

# Problem 8

### Approach

This is similar to the previous question. We just need to add an additional parameter **k**.  
Let F(i, k, j) be the boolean value whether we can form value **j** with the first **i** coins and **k** coins at most:
- F(i, k, j) = F(i-1, k, j) or F(i-1, k-1, j-coins\[i\]) or F(i, k-1, j-coins\[i\]) if j >= coins\[i\]  
- F(i, k, 0) = True  

So we build up the table with bottom-up approach and also we can optimize the space complexity.

### Code

In [190]:
def make_change3(value: int, k: int, coins: List[int]) -> bool:
    dp = [[[False] * (value+1) for _ in range(k+1)] for _ in range(2)]
    for i in range(len(dp)):
        for k in range(len(dp[0])):
            dp[i][k][0] = True
    
    for i in range(1, len(coins)+1):
        curr, prev = i % 2, (i + 1) % 2        
        for k in range(1, len(dp[0])):
            for j in range(1, value+1):
                dp[curr][k][j] = dp[prev][k][j]
                if j - coins[i-1] >= 0:
                    dp[curr][k][j] = dp[curr][k][j] or dp[curr][k-1][j-coins[i-1]] or dp[prev][k-1][j-coins[i-1]]
    
    return dp[len(coins)%2][k][value]    

### Analysis

Time complexity: O(nvk) for the three for loops.  
Space complexity: O(vk) for the 2 * v * k dp array.

<br/><br/>

# Problem 9

### Approach

This is the just like Problem 7, a 0-1 knapsack problem.  
Let F(i, j) be the boolean value whether we can form value **j** with the first **i** numbers:
- F(i, j) = F(i-1, j) or F(i-1, j-nums\[i\]) if j >= nums\[i\]
- F(i, 0) = True  

So we build up the table with bottom-up approach and also we can optimize the space complexity.

### Code

In [202]:
def add_to_t(t: int, nums: List[int]) -> bool:
    dp = [[False] * (t+1) for _ in range(2)]
    for i in range(len(dp)):
        dp[i][0] = True
    
    for i in range(1, len(nums) + 1):
        curr, prev = i % 2, (i + 1) % 2
        for j in range(1, t+1):
            dp[curr][j] = dp[prev][j]
            if j - nums[i-1] >= 0:
                dp[curr][j] = dp[curr][j] or dp[prev][j-nums[i-1]]
                
    return dp[len(nums)%2][t]

### Analysis:

Time complexity: O(nt) for the two for loops.  
Space complexity: O(v) for the 2 * t dp array.